In [16]:
from dotenv import load_dotenv
import os

load_dotenv()  # automatically loads the .env file
HF_TOKEN = os.getenv("HF_TOKEN")

In [17]:
from datasets import load_dataset

dataset = load_dataset("google/code_x_glue_cc_defect_detection")
test_ds = dataset["test"]


In [18]:
from openai import OpenAI

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)


In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model_metrics(test_dataset):
    y_true = []
    y_pred = []
    
    for example in test_dataset:
        prompt = f"Is the following code buggy? Answer only True or False.\n\n{example['func']}"
        completion = client.chat.completions.create(
            model="meta-llama/Llama-3.1-8B-Instruct",
            messages=[{"role": "user", "content": prompt}],
        )
        output = completion.choices[0].message.content.strip()
        
        y_true.append(example["target"])
        # Convert model output to boolean
        y_pred.append(output.lower() == "true")
    
    # Compute metrics
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    
    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "confusion_matrix": cm
    }



In [20]:
metrics = evaluate_model_metrics(test_ds)
print(metrics)

APIStatusError: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}